# 4.2 索引操作

**推荐学习资料：**[Joyful Pandas 第三章](http://joyfulpandas.datawhale.club/Content/ch3.html)

In [1]:
import numpy as np
import pandas as pd

## 1. 表的列索引
列索引是最常见的索引形式，一般通过`[]`来实现。通过`[列名]`可以从`DataFrame`中取出相应的列，返回值为`Series`，例如从表中取出姓名一列：

In [2]:
df = pd.read_csv('learn_pandas.csv', usecols = ['School', 'Grade', 'Name', 'Gender', 'Weight', 'Transfer'])
df['Name'].head()

0      Gaopeng Yang
1    Changqiang You
2           Mei Sun
3      Xiaojuan Sun
4       Gaojuan You
Name: Name, dtype: object

如果要取出多个列，则可以通过`[列名组成的列表]`，其返回值为一个`DataFrame`，例如从表中取出性别和姓名两列：

In [3]:
df[['Gender', 'Name']].head()

,Gender,Name
0,Female,Gaopeng Yang
1,Male,Changqiang You
2,Male,Mei Sun
3,Female,Xiaojuan Sun
4,Male,Gaojuan You


此外，若要取出单列，且列名中不包含空格，则可以用`.列名`取出，这和`[列名]`是等价的：

In [4]:
df.Name.head()

0      Gaopeng Yang
1    Changqiang You
2           Mei Sun
3      Xiaojuan Sun
4       Gaojuan You
Name: Name, dtype: object

## 2. 序列的行索引

### 2.1 以字符串为索引的`Series`

如果取出单个索引的对应元素，则可以使用`[item]`，若`Series`只有单个值对应，则返回这个标量值，如果有多个值对应，则返回一个`Series`：

In [5]:
s = pd.Series([1, 2, 3, 4, 5, 6], index=['a', 'b', 'a', 'a', 'a', 'c'])
s['a']

a    1
a    3
a    4
a    5
dtype: int64

In [6]:
s['b']

2

如果取出多个索引的对应元素，则可以使用`[items的列表]`：

In [7]:
s[['c', 'b']]

c    6
b    2
dtype: int64

如果想要取出某两个索引之间的元素，并且这两个索引是在整个索引中唯一出现，则可以使用切片,，同时需要注意这里的切片会包含两个端点：

In [8]:
s['c': 'b': -2]

c    6
a    4
b    2
dtype: int64

如果前后端点的值重复出现，那么需要经过排序才能使用切片：

In [9]:
try:
    s['a': 'b']
except Exception as e:
    Err_Msg = e
Err_Msg

KeyError("Cannot get left slice bound for non-unique label: 'a'")

In [10]:
s.sort_index()['a': 'b']

a    1
a    3
a    4
a    5
b    2
dtype: int64

### 2.2 以整数为索引的`Series`

在使用数据的读入函数时，如果不特别指定所对应的列作为索引，那么会生成从0开始的整数索引作为默认索引。当然，任意一组符合长度要求的整数都可以作为索引。

和字符串一样，如果使用`[int]`或`[int_list]`，则可以取出对应索引**元素**的值：

In [11]:
s = pd.Series(['a', 'b', 'c', 'd', 'e', 'f'], index=[1, 3, 1, 2, 5, 4])
s[1]

1    a
1    c
dtype: object

In [12]:
s[[2,3]]

2    d
3    b
dtype: object

如果使用整数切片，则会取出对应索引**位置**的值，注意这里的整数切片同`Python`中的切片一样不包含右端点：

In [13]:
s[1:-1:2]

3    b
2    d
dtype: object

## 3. loc索引器

前面讲到了对`DataFrame`的列进行选取，下面要讨论其行的选取。对于表而言，有两种索引器，一种是基于**元素**的`loc`索引器，另一种是基于**位置**的`iloc`索引器。

`loc`索引器的一般形式是`loc[*, *]`，其中第一个`*`代表行的选择，第二个`*`代表列的选择，如果省略第二个位置写作`loc[*]`，这个`*`是指行的筛选。其中，`*`的位置一共有五类合法对象，分别是：单个元素、元素列表、元素切片、布尔列表以及函数，下面将依次说明。

为了演示相应操作，先利用`set_index`方法把`Name`列设为索引，关于该函数的其他用法将在多级索引一章介绍。

In [7]:
df_demo = df.set_index('Name')
df_demo.head()

,School,Grade,Gender,Weight,Transfer
Name,,,,,
Gaopeng Yang,Shanghai Jiao Tong University,Freshman,Female,46.0,N
Changqiang You,Peking University,Freshman,Male,70.0,N
Mei Sun,Shanghai Jiao Tong University,Senior,Male,89.0,N
Xiaojuan Sun,Fudan University,Sophomore,Female,41.0,N
Gaojuan You,Fudan University,Sophomore,Male,74.0,N


### 3.1 `*`为单个元素

此时，直接取出相应的行或列，如果该元素在索引中重复则结果为`DataFrame`，否则为`Series`：

In [8]:
df_demo.loc['Qiang Sun'] # 多个人叫此名字

,School,Grade,Gender,Weight,Transfer
Name,,,,,
Qiang Sun,Tsinghua University,Junior,Female,53.0,N
Qiang Sun,Tsinghua University,Sophomore,Female,40.0,N
Qiang Sun,Shanghai Jiao Tong University,Junior,Female,NaN,N


In [16]:
df_demo.loc['Quan Zhao'] # 名字唯一

School      Shanghai Jiao Tong University
Grade                              Junior
Gender                             Female
Weight                               53.0
Transfer                                N
Name: Quan Zhao, dtype: object

也可以同时选择行和列：

In [17]:
df_demo.loc['Qiang Sun', 'School'] # 返回Series

Name
Qiang Sun              Tsinghua University
Qiang Sun              Tsinghua University
Qiang Sun    Shanghai Jiao Tong University
Name: School, dtype: object

In [18]:
df_demo.loc['Quan Zhao', 'School'] # 返回单个元素

'Shanghai Jiao Tong University'

### 3.2 `*`为元素列表

此时，取出列表中所有元素值对应的行或列：

In [19]:
df_demo.loc[['Qiang Sun','Quan Zhao'], ['School','Gender']]

,School,Gender
Name,,
Qiang Sun,Tsinghua University,Female
Qiang Sun,Tsinghua University,Female
Qiang Sun,Shanghai Jiao Tong University,Female
Quan Zhao,Shanghai Jiao Tong University,Female


### 3.3 `*`为切片

之前的`Series`使用字符串索引时提到，如果是唯一值的起点和终点字符，那么就可以使用切片，并且包含两个端点，如果不唯一则报错：

In [20]:
df_demo.loc['Gaojuan You':'Gaoqiang Qian', 'School':'Gender']

,School,Grade,Gender
Name,,,
Gaojuan You,Fudan University,Sophomore,Male
Xiaoli Qian,Tsinghua University,Freshman,Female
Qiang Chu,Shanghai Jiao Tong University,Freshman,Female
Gaoqiang Qian,Tsinghua University,Junior,Female


需要注意的是，如果`DataFrame`使用整数索引，其使用整数切片的时候和上面字符串索引的要求一致，都是**元素**切片，包含端点且起点、终点不允许有重复值。

In [21]:
df_loc_slice_demo = df_demo.copy()
df_loc_slice_demo.index = range(df_demo.shape[0],0,-1)
df_loc_slice_demo.loc[5:3]

,School,Grade,Gender,Weight,Transfer
5,Fudan University,Junior,Female,46.0,N
4,Tsinghua University,Senior,Female,50.0,N
3,Shanghai Jiao Tong University,Senior,Female,45.0,N


In [22]:
df_loc_slice_demo.loc[3:5] # 没有返回，说明不是整数位置切片

,School,Grade,Gender,Weight,Transfer


### 3.4 `*`为布尔列表

在实际的数据处理中，根据条件来筛选行是极其常见的，此处传入`loc`的布尔列表与`DataFrame`长度相同，且列表为`True`的位置所对应的行会被选中，`False`则会被剔除。

例如，选出体重超过70kg的学生：

In [23]:
df_demo.loc[df_demo.Weight>70].head()

,School,Grade,Gender,Weight,Transfer
Name,,,,,
Mei Sun,Shanghai Jiao Tong University,Senior,Male,89.0,N
Gaojuan You,Fudan University,Sophomore,Male,74.0,N
Xiaopeng Zhou,Shanghai Jiao Tong University,Freshman,Male,74.0,N
Xiaofeng Sun,Tsinghua University,Senior,Male,71.0,N
Qiang Zheng,Shanghai Jiao Tong University,Senior,Male,87.0,N


前面所提到的传入元素列表，也可以通过`isin`方法返回的布尔列表等价写出，例如选出所有大一和大四的同学信息：

In [24]:
df_demo.loc[df_demo.Grade.isin(['Freshman', 'Senior'])].head()

,School,Grade,Gender,Weight,Transfer
Name,,,,,
Gaopeng Yang,Shanghai Jiao Tong University,Freshman,Female,46.0,N
Changqiang You,Peking University,Freshman,Male,70.0,N
Mei Sun,Shanghai Jiao Tong University,Senior,Male,89.0,N
Xiaoli Qian,Tsinghua University,Freshman,Female,51.0,N
Qiang Chu,Shanghai Jiao Tong University,Freshman,Female,52.0,N


对于复合条件而言，可以用`|（或）, &（且）, ~（取反）`的组合来实现，例如选出复旦大学中体重超过70kg的大四学生，或者北大男生中体重超过80kg的非大四的学生：

In [25]:
condition_1_1 = df_demo.School == 'Fudan University'
condition_1_2 = df_demo.Grade == 'Senior'
condition_1_3 = df_demo.Weight > 70
condition_1 = condition_1_1 & condition_1_2 & condition_1_3
condition_2_1 = df_demo.School == 'Peking University'
condition_2_2 = df_demo.Grade == 'Senior'
condition_2_3 = df_demo.Weight > 80
condition_2 = condition_2_1 & (~condition_2_2) & condition_2_3
df_demo.loc[condition_1 | condition_2]

,School,Grade,Gender,Weight,Transfer
Name,,,,,
Qiang Han,Peking University,Freshman,Male,87.0,N
Chengpeng Zhou,Fudan University,Senior,Male,81.0,N
Changpeng Zhao,Peking University,Freshman,Male,83.0,N
Chengpeng Qian,Fudan University,Senior,Male,73.0,Y


### 3.5 `*`为函数

这里的函数，必须以前面的四种合法形式之一为返回值，并且函数的输入值为`DataFrame`本身。假设仍然是上述复合条件筛选的例子，可以把逻辑写入一个函数中再返回，需要注意的是函数的形式参数`x`本质上即为`df_demo`：

In [26]:
def condition(x):
    condition_1_1 = x.School == 'Fudan University'
    condition_1_2 = x.Grade == 'Senior'
    condition_1_3 = x.Weight > 70
    condition_1 = condition_1_1 & condition_1_2 & condition_1_3
    condition_2_1 = x.School == 'Peking University'
    condition_2_2 = x.Grade == 'Senior'
    condition_2_3 = x.Weight > 80
    condition_2 = condition_2_1 & (~condition_2_2) & condition_2_3
    result = condition_1 | condition_2
    return result
df_demo.loc[condition]

,School,Grade,Gender,Weight,Transfer
Name,,,,,
Qiang Han,Peking University,Freshman,Male,87.0,N
Chengpeng Zhou,Fudan University,Senior,Male,81.0,N
Changpeng Zhao,Peking University,Freshman,Male,83.0,N
Chengpeng Qian,Fudan University,Senior,Male,73.0,Y


此外，还支持使用`lambda`表达式，其返回值也同样必须是先前提到的四种形式之一：

In [27]:
df_demo.loc[lambda x:'Quan Zhao', lambda x:'Gender']

'Female'

由于函数无法返回如`start: end: step`的切片形式，故返回切片时要用`slice`对象进行包装：

In [28]:
df_demo.loc[lambda x: slice('Gaojuan You', 'Gaoqiang Qian')]

,School,Grade,Gender,Weight,Transfer
Name,,,,,
Gaojuan You,Fudan University,Sophomore,Male,74.0,N
Xiaoli Qian,Tsinghua University,Freshman,Female,51.0,N
Qiang Chu,Shanghai Jiao Tong University,Freshman,Female,52.0,N
Gaoqiang Qian,Tsinghua University,Junior,Female,50.0,N


最后需要指出的是，对于`Series`也可以使用`loc`索引，其遵循的原则与`DataFrame`中用于行筛选的`loc[*]`完全一致，此处不再赘述。

## 4. iloc索引器

`iloc`的使用与`loc`完全类似，只不过是针对位置进行筛选，在相应的`*`位置处一共也有五类合法对象，分别是：整数、整数列表、整数切片、布尔列表以及函数，函数的返回值必须是前面的四类合法对象中的一个，其输入同样也为`DataFrame`本身。


In [29]:
df_demo.iloc[1, 1] # 第二行第二列

'Freshman'

In [30]:
df_demo.iloc[[0, 1], [0, 1]] # 前两行前两列

,School,Grade
Name,,
Gaopeng Yang,Shanghai Jiao Tong University,Freshman
Changqiang You,Peking University,Freshman


In [31]:
df_demo.iloc[1: 4, 2:4] # 切片不包含结束端点

,Gender,Weight
Name,,
Changqiang You,Male,70.0
Mei Sun,Male,89.0
Xiaojuan Sun,Female,41.0


In [32]:
df_demo.iloc[lambda x: slice(1, 4)] # 传入切片为返回值的函数

,School,Grade,Gender,Weight,Transfer
Name,,,,,
Changqiang You,Peking University,Freshman,Male,70.0,N
Mei Sun,Shanghai Jiao Tong University,Senior,Male,89.0,N
Xiaojuan Sun,Fudan University,Sophomore,Female,41.0,N


在使用布尔列表的时候要特别注意，不能传入`Series`而必须传入序列的`values`，否则会报错。因此，在使用布尔筛选的时候还是应当优先考虑`loc`的方式。

例如，选出体重超过80kg的学生：

In [33]:
df_demo.iloc[(df_demo.Weight>80).values].head()

,School,Grade,Gender,Weight,Transfer
Name,,,,,
Mei Sun,Shanghai Jiao Tong University,Senior,Male,89.0,N
Qiang Zheng,Shanghai Jiao Tong University,Senior,Male,87.0,N
Qiang Han,Peking University,Freshman,Male,87.0,N
Chengpeng Zhou,Fudan University,Senior,Male,81.0,N
Feng Han,Shanghai Jiao Tong University,Sophomore,Male,82.0,N


对`Series`而言同样也可以通过`iloc`返回相应位置的值或子序列：

In [34]:
df_demo.School.iloc[1]

'Peking University'

In [35]:
df_demo.School.iloc[1:5:2]

Name
Changqiang You    Peking University
Xiaojuan Sun       Fudan University
Name: School, dtype: object